<a href="https://colab.research.google.com/github/jackbowley/MMM/blob/main/MeridianTests/Meridian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


### **Running test linear national models through meridian**

### reduce price variable to see impact


<a name="install"></a>

## Step 0: Install


In [2]:
import sys, os
IN_COLAB = ('google.colab' in sys.modules) or ('COLAB_RELEASE_TAG' in os.environ)

In [3]:
# Install meridian: from PyPI @ latest release (robust in Colab and local Jupyter)
import sys, subprocess
pkg = "google-meridian[colab,and-cuda]" if IN_COLAB else "google-meridian"
print(f"Installing: {pkg}")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])
except Exception as e:
    print(f"pip install failed for {pkg}: {e}")

Installing: google-meridian


In [4]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder as data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

if IN_COLAB:
    from google.colab import drive


ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)


Your runtime has 16.9 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


### Functions etc.


In [5]:
def import_df_from_drive(filepath):
  """Imports a CSV file from Google Drive into a pandas DataFrame.

  Args:
    filepath: The full path to the CSV file in Google Drive.

  Returns:
    A pandas DataFrame containing the data from the CSV file, or None if an error
    occurs.
  """
  try:
    drive.mount('/content/drive', force_remount=False) # Force remount in case it's already mounted
    df = pd.read_csv(filepath)
    print(f"DataFrame loaded successfully from {filepath}")
    return df
  except FileNotFoundError:
    print(f"Error: The file was not found at {filepath}")
    return None
  except Exception as e:
    print(f"An error occurred while reading the CSV file: {e}")
    return None


In [6]:
def import_df_from_local(filename, base_dir='results'):
    """Imports a CSV file from a local 'results' folder into a pandas DataFrame.

    Args:
      filename: The CSV file name inside the results folder (e.g., 'data.csv').
      base_dir: Base directory to look in (defaults to 'results').

    Returns:
      A pandas DataFrame containing the data from the CSV file, or None if an error occurs.
    """
    try:
        if not filename:
            raise ValueError("filename must be provided, e.g., 'data.csv'")
        local_path = os.path.join(base_dir, filename)
        if not os.path.isabs(local_path):
            local_path = os.path.abspath(local_path)
        df = pd.read_csv(local_path)
        print(f"DataFrame loaded successfully from {local_path}")
        return df
    except FileNotFoundError:
        print(f"Error: The file was not found at {local_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the CSV file: {e}")
        return None


<a name="load-data"></a>

## Step 1: Load the data


### 1\.1 Load Data


In [7]:

# Define the file path to your CSV file in Google Drive.
if IN_COLAB:
    base_dir = '/content/drive/MyDrive/work/MMM'
    filename = 'data_add.csv'
    df_data = import_df_from_drive(os.path.join(base_dir, filename))
    filename= 'var_spec_add.csv'
    df_var_spec = import_df_from_drive(os.path.join(base_dir, filename))
    df_var_spec.head()
else:
    base_dir = 'results'
    filename = 'data_add.csv'
    df_data = import_df_from_local(filename, base_dir=base_dir)
    filename = 'var_spec_add.csv'
    df_var_spec = import_df_from_local(filename, base_dir=base_dir)

DataFrame loaded successfully from c:\Users\User\repos\MMM\MeridianTests\results\data_add.csv
DataFrame loaded successfully from c:\Users\User\repos\MMM\MeridianTests\results\var_spec_add.csv


In [8]:
df = df_data.copy()
df = df.dropna(subset=['svol_xf'])
df = df.rename(columns={'Date': 'time'})
start_date = '2020-06-02'
end_date = '2024-08-27'
df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]

In [ ]:
## half the price and see the impact
df['price_se_xf'] = df['price_se_xf'] / 2

2\. Load Data using DataFrameInputDataBuilder


In [24]:
builder = data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue',
    default_kpi_column='svol_xf',
    default_revenue_per_kpi_column='price_se_xf',
)

controls = [
    "d_jan","d_feb","d_mar","d_apr","d_may","d_sep","d_oct","d_nov","d_dec",
    "school_easter","school_ht_feb","school_ht_may","school_ht_oct","payday_25",
    "bh_ny","bh_xmas","day_valentine","ww_nat_dlta_maxtemp","ww_nat_dlta_rain",
    "ww_nat_dlta_sun","rsi_nfood_vol_sa","dist_xf",
    "price_se_xf","prom_tft","pos_fsdu","comp_lor_tot","comp_no7_tot"
]

channels = [
    "m_wow_tv","m_wow_olv","m_wow_social","m_amaze_tot","m_celeb_tv","m_celeb_outdoor","m_celeb_display"
]

builder = (
    builder
        .with_kpi(df)
        .with_revenue_per_kpi(df)
        .with_controls(df, control_cols=controls)
)

builder = builder.with_media(
    df,
    media_cols=channels,
    media_spend_cols=channels,
    media_channels=channels,
)

data = builder.build()


<a name="configure-model"></a>

## Step 2: Configure the model


In [ ]:
roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

c:\Users\User\repos\MMM\.venv\Lib\site-packages\meridian\model\model.py:66: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(


In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)

In [ ]:
file_path = '/content/drive/MyDrive/work/MMM/saved_mmm_additive_halfPrice.pkl'
model.save_mmm(mmm, file_path)